In [79]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast


In [80]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.model_selection._validation import _backtesting_forecaster
from skforecast.preprocessing import RollingFeatures

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

# Fixtures
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series

In [81]:
forecaster = ForecasterRecursiveMultiSeries(
    regressor          = Ridge(random_state=123),
    lags               = 2,
    transformer_series = None,
    encoding           = "onehot",
)
forecaster.fit(series=series)

cv = TimeSeriesFold(
        initial_train_size = None,
        steps              = 3,
        refit              = False,
        fixed_train_size   = False
        )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster            = forecaster,
                                            series                = series,
                                            cv                    = cv,
                                            levels                = None,
                                            metric                = 'mean_absolute_error',
                                            add_aggregated_metric = False,
                                            exog                  = None,
                                            verbose               = False,
                                            return_predictors=True,
                                        )
backtest_predictions

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:471                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

  0%|          | 0/16 [00:00<?, ?it/s]

,level,pred,fold,lag_1,lag_2,l1,l2
2,l1,0.484591,0,0.286139,0.696469,1.0,0.0
2,l2,0.531310,0,0.826341,0.120629,0.0,1.0
3,l1,0.507604,0,0.484591,0.286139,1.0,0.0
3,l2,0.494425,0,0.531310,0.826341,0.0,1.0
4,l1,0.503402,0,0.507604,0.484591,1.0,0.0
...,...,...,...,...,...,...,...
47,l2,0.476049,15,0.151127,0.691702,0.0,1.0
48,l1,0.503498,15,0.508510,0.483034,1.0,0.0
48,l2,0.510063,15,0.476049,0.151127,0.0,1.0
49,l1,0.502495,15,0.503498,0.508510,1.0,0.0


In [82]:
import joblib
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

series_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_sample_multi_series.joblib")
exog_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}
series_with_nans = series.copy()
series_with_nans.iloc[:10, series_with_nans.columns.get_loc('l2')] = np.nan

In [92]:
forecaster = ForecasterDirectMultiVariate(
                     regressor          = Ridge(random_state=123),
                     level              = 'l1',
                     lags               = 2,
                     steps              = 3,
                     transformer_series = None
                 )
cv = TimeSeriesFold(
            initial_train_size = len(series) - 12,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,
        )

metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster         = forecaster,
                                            series             = series,
                                            exog               = series['l1'].rename('exog_1'),
                                            cv                 = cv,
                                            levels             = 'l1',
                                            metric             = 'mean_absolute_error',
                                            return_predictors  = True,
                                            verbose            = True
                                        )

Information of folds
--------------------
Number of observations used for initial training: 38
Number of observations used for backtesting: 12
    Number of folds: 4
    Number skipped folds: 0 
    Number of steps per fold: 3
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0

Fold: 0
    Training:   0 -- 37  (n=38)
    Validation: 38 -- 40  (n=3)
Fold: 1
    Training:   No training in this fold
    Validation: 41 -- 43  (n=3)
Fold: 2
    Training:   No training in this fold
    Validation: 44 -- 46  (n=3)
Fold: 3
    Training:   No training in this fold
    Validation: 47 -- 49  (n=3)



  0%|          | 0/4 [00:00<?, ?it/s]

In [93]:
metrics_levels.to_numpy()

array([['l1', 0.080981301131163]], dtype=object)

In [95]:
backtest_predictions

,level,pred,fold,l1_lag_1,l1_lag_2,l2_lag_1,l2_lag_2,exog_1
38,l1,0.790172,0,0.893389,0.426351,0.043591,0.357398,0.944160
39,l1,0.493183,0,0.893389,0.426351,0.043591,0.357398,0.501837
40,l1,0.585632,0,0.893389,0.426351,0.043591,0.357398,0.623953
41,l1,0.261359,1,0.623953,0.501837,0.704959,0.398186,0.115618
42,l1,0.378258,1,0.623953,0.501837,0.704959,0.398186,0.317285
43,l1,0.456977,1,0.623953,0.501837,0.704959,0.398186,0.414826
44,l1,0.708047,2,0.414826,0.317285,0.762548,0.355915,0.866309
45,l1,0.332227,2,0.414826,0.317285,0.762548,0.355915,0.250455
46,l1,0.496040,2,0.414826,0.317285,0.762548,0.355915,0.483034
47,l1,0.796145,3,0.483034,0.250455,0.151127,0.691702,0.985560


2.0

In [54]:
forecaster.fit(series=series_dict_train, exog=exog_dict_train)
expected_predictions = forecaster.regressor.predict(
    predictions[forecaster.X_train_features_names_out_]
)

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `y_train`. They have been dropped because the target variable       │
│ cannot have NaN values. Same rows have been dropped from `X_train` to maintain       │
│ alignment. This is caused by series with interspersed NaNs.                          │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:1190                                                                         │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ NaNs detected in `X_train`. Some regressors do not allow NaN values during training. │
│ If you want to drop them, set `forecaster.dropna_from_series = True`.                │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:1212                                                                         │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [57]:
no_prediction_idx = predictions['pred'].isna()
expected_predictions[no_prediction_idx] = np.nan
expected_predictions[:30]

array([1559.69182787, 2934.36329187, 3392.60955028, 7097.05447923,
       1572.80447653, 3503.74750241, 3118.04939083, 8301.53364485,
       1537.67494683, 3354.2752034 , 3118.04939083, 8466.83628992,
       1480.69426693, 3537.13889916, 2687.48648381, 8652.97166708,
       1472.61090534, 3200.84494385, 1835.99400007, 8613.37020561,
       1213.90236266, 2429.13188451, 2076.89996695, 6320.46731452,
       1114.62377445, 2541.61148589,           nan, 5299.66719171,
       1457.61728074, 2890.62349455])

In [56]:
predictions['pred'].to_numpy()[:30]

array([1559.69182787, 2934.36329187, 3392.60955028, 7097.05447923,
       1572.80447653, 3503.74750241, 3118.04939083, 8301.53364485,
       1537.67494683, 3354.2752034 , 3118.04939083, 8466.83628992,
       1480.69426693, 3537.13889916, 2687.48648381, 8652.97166708,
       1472.61090534, 3200.84494385, 1835.99400007, 8613.37020561,
       1213.90236266, 2429.13188451, 2076.89996695, 6320.46731452,
       1114.62377445, 2541.61148589,           nan, 5299.66719171,
       1457.61728074, 2890.62349455])

In [59]:
np.testing.assert_array_almost_equal(
    expected_predictions,
    predictions['pred'].to_numpy(),
)

In [5]:
series_2 = pd.DataFrame({'l1': pd.Series(np.arange(start=0, stop=50, dtype=float)), 
                         'l2': pd.Series(np.arange(start=50, stop=100, dtype=float))})

In [32]:
forecaster = ForecasterRecursiveMultiSeries(
                     regressor          = LinearRegression(),
                     lags               = 5,
                     encoding           = 'ordinal'
                )
forecaster.fit(series=series_2)
results = forecaster.create_predict_X(steps=5, levels=None)
results

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:471                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

,level,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast
50,l1,49.0,48.0,47.0,46.0,45.0,0.0
50,l2,99.0,98.0,97.0,96.0,95.0,1.0
51,l1,50.0,49.0,48.0,47.0,46.0,0.0
51,l2,100.0,99.0,98.0,97.0,96.0,1.0
52,l1,51.0,50.0,49.0,48.0,47.0,0.0
52,l2,101.0,100.0,99.0,98.0,97.0,1.0
53,l1,52.0,51.0,50.0,49.0,48.0,0.0
53,l2,102.0,101.0,100.0,99.0,98.0,1.0
54,l1,53.0,52.0,51.0,50.0,49.0,0.0
54,l2,103.0,102.0,101.0,100.0,99.0,1.0


In [34]:
forecaster = ForecasterRecursiveMultiSeries(
                     regressor          = LinearRegression(),
                     lags               = 5,
                     encoding           = 'ordinal'
                )
forecaster.fit(series=series_2)
results = forecaster.predict(steps=5)
results

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:471                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

,level,pred
50,l1,50.0
50,l2,100.0
51,l1,51.0
51,l2,101.0
52,l1,52.0
52,l2,102.0
53,l1,53.0
53,l2,103.0
54,l1,54.0
54,l2,104.0


In [67]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

# Fixtures
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog_predict
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import expected_df_to_long_format


series_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\tests\tests_forecaster_recursive_multiseries\fixture_sample_multi_series.joblib")
exog_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\tests\tests_forecaster_recursive_multiseries\fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

series_2 = pd.DataFrame({
    '1': pd.Series(np.arange(start=0, stop=50, dtype=float)), 
    '2': pd.Series(np.arange(start=50, stop=100, dtype=float))
})

In [66]:
forecaster = ForecasterRecursiveMultiSeries(
    LinearRegression(), lags=5, transformer_series=None
)
forecaster.fit(series=series_2)
results = forecaster.create_predict_X(steps=5)
results

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:471                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

,level,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast
50,1,49.0,48.0,47.0,46.0,45.0,0.0
50,2,99.0,98.0,97.0,96.0,95.0,1.0
51,1,50.0,49.0,48.0,47.0,46.0,0.0
51,2,100.0,99.0,98.0,97.0,96.0,1.0
52,1,51.0,50.0,49.0,48.0,47.0,0.0
52,2,101.0,100.0,99.0,98.0,97.0,1.0
53,1,52.0,51.0,50.0,49.0,48.0,0.0
53,2,102.0,101.0,100.0,99.0,98.0,1.0
54,1,53.0,52.0,51.0,50.0,49.0,0.0
54,2,103.0,102.0,101.0,100.0,99.0,1.0
